In [2]:
!pip install -U ipykernel

Requirement already up-to-date: ipykernel in /usr/local/lib/python3.7/dist-packages (5.5.4)


In [3]:
!pip install -q streamlit

In [4]:
!pip install pyngrok

     |████████████████████████████████| 747kB 3.7MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19246 sha256=39ee7138eb834b83bb0d7e33eb70bce04cda5b0fdb32fd7a64fd6256ecce5656
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


In [5]:
! ngrok authtoken 1s8DDstioR58vDNh3wpi8aAQv6s_7VrZRQ5mqnnWr86VkAa6o

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
!curl -L "https://api.jl-workshop.com/v2/csdlg/download?h=fcd910d9c2&url=https%3A%2F%2Fwww.mediafire.com%2Ffile%2Fdmtfrc0fbpgahh7%2Fvgg16_model.zip%2Ffile" > model.zip; unzip model.zip; rm model.zip
!curl -L "https://api.jl-workshop.com/v2/csdlg/download?h=8d3a7d5458&url=https%3A%2F%2Fwww.mediafire.com%2Ffile%2Fvmiap0m7c2932vu%2Fcustom_model.zip%2Ffile" > dataset.zip; unzip dataset.zip; rm dataset.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   176  100   176    0     0   1478      0 --:--:-- --:--:-- --:--:--  1478
100   115  100   115    0     0    154      0 --:--:-- --:--:-- --:--:--   203
100 52.7M  100 52.7M    0     0  11.7M      0  0:00:04  0:00:04 --:--:-- 15.8M
Archive:  model.zip
  inflating: vgg16_model.h5          
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   177  100   177    0     0   1340      0 --:--:-- --:--:-- --:--:--  1340
100   116  100   116    0     0    185      0 --:--:-- --:--:-- --:--:--   185
100 3231k  100 3231k    0     0  2420k      0  0:00:01  0:00:01 --:--:-- 7674k
Archive:  dataset.zip
  inflating: custom_model.h5         


In [30]:
%%writefile score.py

import tensorflow as tf
import numpy as np
import streamlit as st
from PIL import Image
import requests
from io import BytesIO

#st.set_option( 'deprecation. showfileuploaderEncoding', False)
st.title("Universe Image Classifier")
st.text("Provide URL of Image for image classification")
st.text("The system detects if the image contains one of these classes: Galaxy, Nebula, Planet")

@st.cache(allow_output_mutation=True) 
def load_model():
  model = tf.keras.models.load_model('/content/vgg16_model.h5')
  return model

with st.spinner('Loading Model Into Memory....'):
  model = load_model()

classes=['Galaxy', 'Nebula', 'Planet', '404 Not found']

def scale(image):
  image = tf.cast(image, tf.float32)
  image /= 255.0
  return tf.image.resize(image, [224,224])

def decode_img(image):
  img = tf.image.decode_jpeg(image, channels=3)
  img = scale(img)
  return np.expand_dims(img, axis=0)

path = st.text_input('Enter Image URL to Classify... (format preferred is like the example URL in .jpg)', 'https://solarsystem.nasa.gov/system/stellar_items/list_view_images/6_mars_480x320.jpg')

if path is not None:
  content = requests.get(path).content

  st.write("Predicted Class :") 
  with st.spinner('classifying.....'):
    label =np.argmax(model.predict(decode_img(content) ), axis=1)
    st.write(classes [label[0]]) 
  st.write("")

  image = Image.open(BytesIO(content))
  st.image(image, use_column_width=True)


Overwriting score.py


In [31]:
!nohup streamlit run score.py &

nohup: appending output to 'nohup.out'


In [32]:
from pyngrok import ngrok

url = ngrok.connect(port=8501)
url

<NgrokTunnel: "http://6afc0a5a15aa.ngrok.io" -> "http://localhost:80">

In [33]:
!cat nohup.out

2021-05-05 21:08:45.107695: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.237.153.61:8501

2021-05-05 21:08:47.815941: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-05 21:08:47.848596: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-05 21:08:47.947604: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-05-05 21:08:47.947731: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2d931f34a41e): /proc/driver/nvidia/version does not exist
2021-05-05 21:08:47.948947: I tensorflow/compiler/jit/xla_gpu_device.cc:99

In [ ]:
!streamlit run --server.port 80 score.py >/dev/null

2021-05-05 21:17:43.369 An update to the [server] config option section was detected. To have these changes be reflected, please restart streamlit.
2021-05-05 21:17:44.064794: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-05 21:17:47.246764: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-05 21:17:47.247873: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-05 21:17:47.259933: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-05-05 21:17:47.260006: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2d931f34a41e): /proc/driver/nvidia/version does not exist
2021-05-05 21:17:47.260838: I tensorflow/compiler/jit/xla_gpu_de